### Setup and Configuration

Deploying the agent to AgentCore Runtime

In [ ]:
from bedrock_agentcore_starter_toolkit import Runtime
import boto3
import os
import random, string

session = boto3.Session()
region = session.region_name
print(f"current region: {region}")
account_id = session.client("sts").get_caller_identity()["Account"]
print(f"current account: {account_id}")


### Create IAM Execution Role

The AgentCore runtime requires an IAM role with specific permissions to:
- Access Bedrock models for AI inference
- Retrieve from knowledge bases (KB)
- Write to CloudWatch logs
- Access SSM Parameter for KB ID
- Pull/push Docker images to ECR

We'll create this role using our predefined policy templates that include all necessary permissions.

In [ ]:
# Load policy from external file
with open('policy.json', 'r') as f:
    policy_template = f.read()

# load trust policy from external file
with open('trust-policy.json', 'r') as f:
    trust_policy_template = f.read()

policy = policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id).replace('REPO_ARN', '*')
print(f"Policy loaded and updated with region: {region}, account: {account_id}")

trust_policy = trust_policy_template.replace('REGION', region).replace('ACCOUNT_ID', account_id)

# create IAM role using the policies
suffix = random.choices(string.ascii_lowercase + string.digits, k=8)
iam_client = boto3.client('iam')
role_name = f"bedrock-runtime-execution-role-{''.join(suffix)}"

role = iam_client.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=trust_policy
)

iam_client.put_role_policy(
    RoleName=role_name,
    PolicyName='bedrock-runtime-execution-policy',
    PolicyDocument=policy
)

### Configure Agent Deployment

The `configure()` method sets up the deployment infrastructure:


**What happens during configuration:**
- Creates ECR repository and IAM role (if needed)
- Generates Dockerfile and .dockerignore for containerization
- Creates `.bedrock_agentcore.yaml` configuration file

In [ ]:
agentcore_runtime = Runtime()
agent_name = "mortgage_assistant"
response = agentcore_runtime.configure(
    entrypoint="mortgage_agent_runtime_tools.py",
    execution_role=role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name
)

### Launch Agent Runtime

The `launch()` method triggers a CodeBuild job that:

1. **Pre-build**: Authenticates with ECR registry
2. **Build**: Creates ARM64 Docker image with your agent code
3. **Post-build**: Pushes image to ECR and creates a Bedrock AgentCore runtime and deploys the agent

In [ ]:
launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)

Lets check the status of the agentcore runtime

In [ ]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
print(status)

Time to invoke the agent

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "I am an existing customer, Here is my id :2445566. How much could I save by switching to fortnightly payments "})

And parse the response message

In [ ]:
from IPython.display import Markdown, display
import json
#  Handle multiple response chunks
full_response = b""
for chunk in invoke_response['response']:
    full_response += chunk

response_text = full_response.decode("utf-8")
response_data = json.loads(response_text)
content = response_data['result']['content'][0]['text']
display(Markdown(content))

### Summary
You've successfully enhanced your mortgage assistant with AgentCore's built-in Code Interpreter tool:

- Executes dynamic Python code in secure sandbox environments
- Performs complex mortgage calculations comparing payment strategies
- Generates code on-demand based on customer-specific loan parameters
- Maintains security through isolated code execution
- Provides detailed analysis with formatted currency values and recommendations
- Integrates seamlessly with existing Strands agent architecture